In [0]:
import numpy as np
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 
from gensim.models import Word2Vec

In [0]:
nltk.download('all')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
datapath = 'gdrive/My Drive/data/'

In [0]:
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt

In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    document=[pos for (word,pos) in nltk.pos_tag(tokens)]
    feature_docs.append(document)
  return feature_docs

In [0]:
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
# tr_st = sent_to_features(tr_st)
# val_st = sent_to_features(val_st)



NameError: ignored

In [0]:
from gensim.models import Word2Vec
#define training data
sentences = sent_to_features(np.concatenate((tr_st,val_st)))
# train model
model = Word2Vec(sentences, min_count=1, size=300, window=5)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['NN'])
# save model
model.save(datapath+'model_300d.bin')
# load model
new_model = Word2Vec.load(datapath+'model_300d.bin')
print(new_model)
words = list(new_model.wv.vocab)
# print(words)
print(new_model['NN'])


Word2Vec(vocab=33, size=300, alpha=0.025)
['IN', 'PRP', 'RB', 'VB', 'DT', 'JJ', 'NNS', 'CD', 'NN', 'MD', 'PRP$', 'CC', 'VBD', 'VBG', 'VBN', 'VBZ', 'WDT', 'JJS', 'TO', 'JJR', 'VBP', 'RP', 'RBR', 'WP$', 'WP', 'WRB', 'EX', 'PDT', 'RBS', '$', 'FW', 'NNP', 'NNPS']
[ 4.88917204e-03  2.67336704e-02 -1.15061797e-01 -4.76854928e-02
  6.31363317e-02  5.70424087e-03 -6.50066212e-02 -5.63471317e-02
 -8.57375115e-02 -3.59895616e-03 -7.12884888e-02  8.04653578e-03
  2.00573336e-02 -8.85402411e-02  1.17866553e-01  5.49756773e-02
  7.50548691e-02 -9.14690271e-03 -2.93339472e-02  9.71581712e-02
 -4.72441651e-02  3.95180508e-02 -8.70295707e-03  3.72647494e-02
  2.04304494e-02  4.52256873e-02  2.76430547e-02  7.90186077e-02
 -1.31163970e-02 -4.98526692e-02  1.59276113e-01 -1.26377165e-01
  1.23411268e-01  6.45740479e-02  1.07516780e-01  3.36343758e-02
  2.80246977e-02  1.01473860e-01 -3.19992751e-02  6.72889575e-02
 -1.28998263e-02 -4.87482361e-02  3.80126350e-02 -7.25858584e-02
  8.97195339e-02 -4.23929

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
new_model = Word2Vec.load(datapath+'model_400d.bin')
print(new_model)
words = list(new_model.wv.vocab)
# print(words)
print(new_model['NN'])

Word2Vec(vocab=33, size=400, alpha=0.025)
[ 1.50699390e-03  2.66283918e-02 -1.18310325e-01 -4.03361060e-02
  6.43535107e-02  8.26547481e-03 -6.24405779e-02 -5.37020825e-02
 -8.00247341e-02 -9.65688610e-04 -5.99348769e-02  2.74895388e-03
  1.72228534e-02 -7.91346952e-02  1.10319301e-01  5.00443354e-02
  6.05499893e-02 -5.24658151e-03 -3.16545703e-02  8.37556273e-02
 -3.99062335e-02  3.68105322e-02 -9.24068317e-03  3.78350057e-02
  1.99674014e-02  3.76245491e-02  2.50949189e-02  6.83679804e-02
 -1.11436564e-02 -4.15621810e-02  1.41192183e-01 -1.13539599e-01
  1.06054842e-01  5.75230569e-02  9.36304629e-02  2.40109377e-02
  2.16037296e-02  9.10005867e-02 -2.97344364e-02  5.68358414e-02
 -1.12782428e-02 -4.36322875e-02  3.70878652e-02 -6.16422482e-02
  7.87576288e-02 -3.58052291e-02  1.74845345e-02  3.98569852e-02
 -5.30381221e-03 -5.86291850e-02  4.35798913e-02 -1.54701248e-01
 -3.78588662e-02  5.54725900e-03 -2.15781927e-02 -5.73952943e-02
  5.94663806e-02 -4.63951826e-02  5.33087328e-02

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
from tqdm import tqdm

f = open(datapath+'vecs.txt',encoding='utf-8', errors='ignore')
embedding_values = {}
vectors=[]
embedding_size=400
i=0;
for line in tqdm(f):
  line= re.sub(r'\s+?$', '', line)
  vectors.append(line)

418129it [00:59, 7071.62it/s]


In [0]:
vectors=[]
for word in words:
  v=' '.join(list(map(str,new_model[word])))
  vec=' '.join([word,v])
  vectors.append(vec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
print(vectors[2])

RB -0.001677966 0.024075089 -0.13713928 -0.03391111 0.06626998 0.01625984 -0.064352706 -0.052657254 -0.094835795 0.0033594416 -0.05425444 -0.0044531743 0.010256787 -0.08561474 0.118937746 0.05515573 0.052052144 0.00303764 -0.046000466 0.08531793 -0.03354428 0.041392893 -0.018166084 0.045939833 0.018846637 0.043149773 0.024167651 0.058056552 -0.009557627 -0.046163358 0.15061858 -0.12147938 0.11840858 0.06648922 0.106070094 0.019116294 0.0139758885 0.10919348 -0.03323298 0.060440976 -0.0112360995 -0.04438262 0.048948802 -0.070293956 0.08226813 -0.032325443 0.010559378 0.04023999 -0.0006354818 -0.063054904 0.039290052 -0.1707437 -0.034855187 -0.0033821382 -0.027364388 -0.053812314 0.06437732 -0.03510805 0.048358686 -0.027967019 0.046683054 0.052369688 0.020649422 -0.09546832 -0.023119736 -0.13974944 -0.08145653 0.14328085 -0.18137228 -0.036696732 -0.114251934 -0.0189149 -0.025562478 -0.009946554 0.11961451 0.15641548 -0.059031032 0.008261232 -0.007857856 0.05259373 0.06002585 0.071868144 

In [0]:
f= open(datapath+'vecs-ex.txt','w')
f.write('\n'.join(vectors))